In [32]:
from typing import Any
from datetime import datetime
import json

import openpyxl

In [27]:
def parse_excel_report(file_path) -> list[Any]:
    wb = openpyxl.load_workbook(file_path)
    ws = wb.active
    if not ws:
        raise ValueError('No active worksheet found in the workbook.')

    links = []
    for row in ws.iter_rows():
        result = None
        for idx, cell in enumerate(row):
            if idx == 2 and cell.value == 'Record No.:':
                result = {}
            if idx == 5 and result is not None:
                result['record'] = cell.value
                try:
                    result['link'] = cell.hyperlink.target if cell.hyperlink else None
                except AttributeError:
                    pass
            if idx == 10 and result is not None:
                dt = cell.value
                if isinstance(dt, datetime):
                    result['received_date'] = dt.strftime('%Y-%m-%d')
        
        if result is not None:
            links.append(result)
    return links

In [28]:
records = []
for year in ['2021', '2022', '2023', '2024', '2025']:
    records_this_year = parse_excel_report(f'../data/01/pds_{year}.xlsx')
    records.extend(records_this_year)

In [30]:
# Keep only records that begin with `CUP`
cup_records = [record for record in records if record['record'].startswith('CUP')]

In [ ]:
# Write cup_records to a json file
with open('../data/01/cup_records.json', 'w') as f:
    json.dump(cup_records, f, indent=4)

In [33]:
# What are all the various first 3-letter prefixes of the records?
prefixes = set()
for record in records:
    prefix = record['record'][:3]
    prefixes.add(prefix)
prefixes

{'CAR', 'CUP', 'DRH'}

Unfortunately, variances apparently weren't included in this report, so we'd have to use the search interface for those rather than parsing the Excel sheets.